In [97]:
# Import
import pandas as pd
import numpy as np
import hvplot.pandas
import lib
from pathlib import Path

from pandas.tseries.offsets import DateOffset

# Setting these options will allow for reviewing more of the DataFrames
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)

# Import the finta Python library and the TA module
from finta import TA

## Import the CSV file and create the Pandas DataFrame

In [98]:
# Read in CSV file in from the resources folder into a Pandas DataFrame
# Set the date as the DateTimeIndex
df = pd.read_csv(
    Path("../data/SP500_Data.csv"), 
    index_col="Date",
    infer_datetime_format=True, 
    parse_dates=True
    
)

# Review the DataFrame
df.head()

,Price,Open,High,Low,Change %
Date,,,,,
2002-07-03,954.0,948.1,954.3,934.9,0.62%
2002-07-05,989.0,965.8,989.1,954.0,3.67%
2002-07-08,977.0,989.0,993.6,972.9,-1.21%
2002-07-09,952.8,977.0,979.6,951.7,-2.48%
2002-07-10,920.5,952.8,956.3,920.3,-3.39%


In [99]:
# Plot the DataFrame with hvplot
# Calculate the daily returns using the closing prices and the pct_change function
df["actual_returns"] = df["Price"].pct_change()
df["actual_returns"].hvplot()

:Curve   [Date]   (actual_returns)

### Create a new trading algorithm using the Bollinger Bands technical indicator from the finta library. 

In [100]:
# Create a new clean copy of the last 5 years data

short_testing_begin = df.index.max() - DateOffset(years=5)
bb_signals_df = df.loc[short_testing_begin:]
 

In [101]:
bb_signals_df.columns=["close","open","high","low","change %", "actual_returns"]
# Review the DataFrame
bb_signals_df.head()

,close,open,high,low,change %,actual_returns
Date,,,,,,
2017-05-11,2394.44,2394.84,2395.72,2381.74,-0.22%,-0.002163
2017-05-12,2390.90,2392.44,2392.44,2387.19,-0.15%,-0.001478
2017-05-15,2402.32,2393.98,2404.05,2393.94,0.48%,0.004776
2017-05-16,2400.67,2404.55,2405.77,2396.05,-0.07%,-0.000687
2017-05-17,2357.03,2382.95,2384.87,2356.21,-1.82%,-0.018178


In [102]:
# Determine the Bollinger Bands for the Dataset
bbands_df = TA.BBANDS(bb_signals_df, period=30)

# Review the DataFrame
bbands_df.head()

,BB_UPPER,BB_MIDDLE,BB_LOWER
Date,,,
2017-05-11,NaN,NaN,NaN
2017-05-12,NaN,NaN,NaN
2017-05-15,NaN,NaN,NaN
2017-05-16,NaN,NaN,NaN
2017-05-17,NaN,NaN,NaN


In [103]:
# Concatenate the Bollinger Bands to the DataFrame
bb_signals_df = pd.concat([bb_signals_df, bbands_df], axis=1)

# Review the DataFrame
bb_signals_df.iloc[50:60, :]

,close,open,high,low,change %,actual_returns,BB_UPPER,BB_MIDDLE,BB_LOWER
Date,,,,,,,,,
2017-07-24,2469.91,2472.04,2473.10,2466.32,-0.11%,-0.001064,2475.406775,2440.789667,2406.172558
2017-07-25,2477.13,2477.88,2481.24,2474.91,0.29%,0.002923,2479.151852,2442.381000,2405.610148
2017-07-26,2477.83,2479.97,2481.69,2474.94,0.03%,0.000283,2482.596940,2443.630333,2404.663726
2017-07-27,2475.42,2482.76,2484.04,2459.93,-0.10%,-0.000973,2485.461413,2444.880333,2404.299253
2017-07-28,2472.10,2469.12,2473.53,2464.66,-0.13%,-0.001341,2487.680772,2446.201667,2404.722561
2017-07-31,2470.30,2475.94,2477.96,2468.53,-0.07%,-0.000728,2489.520582,2447.440000,2405.359418
2017-08-01,2476.35,2477.10,2478.51,2471.14,0.24%,0.002449,2491.546383,2448.203000,2404.859617
2017-08-02,2477.57,2480.38,2480.38,2466.48,0.05%,0.000493,2493.970867,2449.554333,2405.137800
2017-08-03,2472.16,2476.03,2476.03,2468.85,-0.22%,-0.002184,2495.609609,2450.772667,2405.935724


## Create a trading algorithm using Bollinger Bands

In [104]:
# Set the Signal column
bb_signals_df["Signal"] = 0.0

# Create a value to hold the initial trade signal
trade_signal = 0

# Update the DataFrame Signal column 1 (entry) or -1 (exit) for a long position trading algorithm
# where 1 is when the Close price is less than the BB_LOWER window
# where -1 is when the Close price is greater the the BB_UPPER window
# Incorporate a conditional in the if-statement, to evaluate the value of the trade_signal so the algorithm 
# plots only 1 entry and exit point per cycle.
for index, row in bb_signals_df.iterrows():
    if (row["close"] < row["BB_LOWER"]) and (trade_signal < 1):
        bb_signals_df.loc[index, "Signal"] = 1.0
        trade_signal += 1
        
    if (row["close"] > row["BB_UPPER"]) and (trade_signal > 0):
        bb_signals_df.loc[index, "Signal"] = -1.0
        trade_signal = 0


# Review the DataFrame
bb_signals_df

,close,open,high,low,change %,actual_returns,BB_UPPER,BB_MIDDLE,BB_LOWER,Signal
Date,,,,,,,,,,
2017-05-11,2394.44,2394.84,2395.72,2381.74,-0.22%,-0.002163,NaN,NaN,NaN,0.0
2017-05-12,2390.90,2392.44,2392.44,2387.19,-0.15%,-0.001478,NaN,NaN,NaN,0.0
2017-05-15,2402.32,2393.98,2404.05,2393.94,0.48%,0.004776,NaN,NaN,NaN,0.0
2017-05-16,2400.67,2404.55,2405.77,2396.05,-0.07%,-0.000687,NaN,NaN,NaN,0.0
2017-05-17,2357.03,2382.95,2384.87,2356.21,-1.82%,-0.018178,NaN,NaN,NaN,0.0
2017-05-18,2365.72,2354.69,2375.74,2352.72,0.37%,0.003687,NaN,NaN,NaN,0.0
2017-05-19,2381.73,2371.37,2389.06,2370.43,0.68%,0.006767,NaN,NaN,NaN,0.0
2017-05-22,2394.02,2387.21,2395.46,2386.92,0.52%,0.005160,NaN,NaN,NaN,0.0
2017-05-23,2398.42,2397.04,2400.85,2393.88,0.18%,0.001838,NaN,NaN,NaN,0.0


In [105]:
# Visualize entry position relative to close price
entry = bb_signals_df[bb_signals_df["Signal"] == 1.0]["close"].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = bb_signals_df[bb_signals_df["Signal"] == -1.0]["close"].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = bb_signals_df[["close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_upper = bb_signals_df[["BB_UPPER"]].hvplot(
    line_color='purple',
    ylabel='Price in $',
    width=1000,
    height=400
)


bb_middle = bb_signals_df[["BB_MIDDLE"]].hvplot(
    line_color='orange',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_lower = bb_signals_df[["BB_LOWER"]].hvplot(
    line_color='blue',
    ylabel='Price in $',
    width=1000,
    height=400
)


# Overlay plots
bbands_plot = security_close * bb_upper * bb_middle * bb_lower * entry * exit
bbands_plot

:Overlay
   .Curve.I          :Curve   [Date]   (close)
   .Curve.II         :Curve   [Date]   (BB_UPPER)
   .Curve.III        :Curve   [Date]   (BB_MIDDLE)
   .Curve.IV         :Curve   [Date]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [Date]   (close)
   .Scatter.Close.II :Scatter   [Date]   (close)

## Backtest the Trading Strategy with 10 share and 100K initial capital

In [106]:
signals_df = bb_signals_df.copy()
# Set the initial capital
initial_capital = float(100000)

# Set the share size
share_size = 10

# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df["Position"] = share_size * signals_df["Signal"]

# Find the points in time where a 100 share position is bought or sold
signals_df["Entry/Exit Position"] = signals_df["Position"]

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df["Portfolio Holdings"] = (
    signals_df["close"] * signals_df["Entry/Exit Position"].cumsum()
)

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df["Portfolio Cash"] = (
    initial_capital - (signals_df["close"] * signals_df["Entry/Exit Position"]).cumsum()
)

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df["Portfolio Total"] = (
    signals_df["Portfolio Cash"] + signals_df["Portfolio Holdings"]
)

# Calculate the portfolio daily returns
signals_df["strategy_returns"] = signals_df["Portfolio Total"].pct_change()

# Calculate the cumulative returns
signals_df["cumulative_Returns"] = (
    1 + signals_df["strategy_returns"]
).cumprod() - 1

# Print the DataFrame
signals_df.head()
#signals_df.to_csv("returns.csv")

In [107]:
# Visualize exit positions relative to total portfolio value
entry = signals_df[signals_df["Signal"] == 1.0]["Portfolio Total"].hvplot.scatter(
    color='purple',
    marker='^',
    legend=False, 
    ylabel="Total Portfolio Value", 
    width=1000, 
    height=400
)

# Visualize entry positions relative to total portfolio value
exit = signals_df[signals_df["Signal"] == -1.0]["Portfolio Total"].hvplot.scatter(
    color='red',
    marker='v',
    legend=False, 
    ylabel="Total Portfolio Value", 
    width=1000, 
    height=400
)

# Visualize the total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay the plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="Bollinger Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

In [108]:
signals_df["Portfolio Total"].hvplot()

:Curve   [Date]   (Portfolio Total)

In [109]:
(1 + signals_df[["actual_returns", "strategy_returns"]]).cumprod().hvplot()

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [110]:
portfolio_evaluation_df = lib.performance_metrics(signals_df, "Bollinger_100K")

portfolio_evaluation_df.head()

,Bollinger_100K
Annualized Return,0.02638
Cumulative Returns,0.13419
Annual Volatility,0.798403
Sharpe Ratio,0.544434
Sortino Ratio,0.751658


In [111]:
# Calculate weekly returns
weekly_returns = signals_df["cumulative_Returns"].groupby(
        by=[
            signals_df.index.year,
            signals_df.index.week
        ]
    ).cumsum()

# Display results    
weekly_returns.head()

C:\Users\padma\anaconda3\envs\algotrading\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  """


Date
2017-05-11    NaN
2017-05-12    0.0
2017-05-15    0.0
2017-05-16    0.0
2017-05-17    0.0
Name: cumulative_Returns, dtype: float64

In [112]:
weekly_returns.hvplot()

:Curve   [Date]   (cumulative_Returns)

In [113]:
weekly_returns.describe()

count    1259.000000
mean        0.194228
std         0.186709
min        -0.027499
25%         0.052838
50%         0.129103
75%         0.289664
max         0.798739
Name: cumulative_Returns, dtype: float64

In [114]:
bb_signals_df.head()

,close,open,high,low,change %,actual_returns,BB_UPPER,BB_MIDDLE,BB_LOWER,Signal
Date,,,,,,,,,,
2017-05-11,2394.44,2394.84,2395.72,2381.74,-0.22%,-0.002163,NaN,NaN,NaN,0.0
2017-05-12,2390.90,2392.44,2392.44,2387.19,-0.15%,-0.001478,NaN,NaN,NaN,0.0
2017-05-15,2402.32,2393.98,2404.05,2393.94,0.48%,0.004776,NaN,NaN,NaN,0.0
2017-05-16,2400.67,2404.55,2405.77,2396.05,-0.07%,-0.000687,NaN,NaN,NaN,0.0
2017-05-17,2357.03,2382.95,2384.87,2356.21,-1.82%,-0.018178,NaN,NaN,NaN,0.0


In [115]:
predictions_df = pd.DataFrame(index=bb_signals_df.index)
predictions_df["predicted_signal"] = bb_signals_df["Signal"]
predictions_df["actual_returns"] = bb_signals_df["actual_returns"]
# Calculate the  strategy returns
predictions_df["strategy_returns"] = predictions_df["actual_returns"] * predictions_df["predicted_signal"].cumsum()
# Calculate the cumulative returns
predictions_df["cumulative_Returns"] = (
    1 + predictions_df["strategy_returns"]
).cumprod() -1

predictions_df.head()

,predicted_signal,actual_returns,strategy_returns,cumulative_Returns
Date,,,,
2017-05-11,0.0,-0.002163,-0.0,0.0
2017-05-12,0.0,-0.001478,-0.0,0.0
2017-05-15,0.0,0.004776,0.0,0.0
2017-05-16,0.0,-0.000687,-0.0,0.0
2017-05-17,0.0,-0.018178,-0.0,0.0


In [116]:
portfolio_evaluation_df = lib.performance_metrics(predictions_df, "Strategy_Bollinger_S_Term")

portfolio_evaluation_df.head()

,Strategy_Bollinger_S_Term
Annualized Return,0.035864
Cumulative Returns,0.105172
Annual Volatility,1.236514
Sharpe Ratio,0.201926
Sortino Ratio,0.277673


In [117]:
(1 + predictions_df[["actual_returns", "strategy_returns"]]).cumprod().hvplot()

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [118]:
portfolio_evaluation_df.to_csv("../data/Strategy_Bollinger_S_Term_Metrics.csv")

In [119]:
predictions_df.to_csv("../data/Strategy_Bollinger_S_Term_Returns.csv")

## Testing against long dataset

In [120]:
long_testing_begin = df.index.min() + DateOffset(years=4)
bb_signals_df = df.loc[long_testing_begin:]
bb_signals_df.columns=["close","open","high","low","change %", "actual_returns"]
# Review the DataFrame
bb_signals_df.head()

,close,open,high,low,change %,actual_returns
Date,,,,,,
2006-07-03,1280.19,1270.06,1280.38,1270.06,0.79%,0.007865
2006-07-05,1270.91,1280.05,1280.05,1265.91,-0.72%,-0.007249
2006-07-06,1274.08,1270.58,1278.32,1270.58,0.25%,0.002494
2006-07-07,1265.48,1274.08,1275.38,1263.13,-0.67%,-0.006750
2006-07-10,1267.34,1265.46,1274.06,1264.46,0.15%,0.001470


In [121]:
# Determine the Bollinger Bands for the Dataset
bbands_df = TA.BBANDS(bb_signals_df, period=30)

# Review the DataFrame
bbands_df.head()

,BB_UPPER,BB_MIDDLE,BB_LOWER
Date,,,
2006-07-03,NaN,NaN,NaN
2006-07-05,NaN,NaN,NaN
2006-07-06,NaN,NaN,NaN
2006-07-07,NaN,NaN,NaN
2006-07-10,NaN,NaN,NaN


In [122]:
# Concatenate the Bollinger Bands to the DataFrame
bb_signals_df = pd.concat([bb_signals_df, bbands_df], axis=1)

# Review the DataFrame
bb_signals_df.iloc[50:60, :]

,close,open,high,low,change %,actual_returns,BB_UPPER,BB_MIDDLE,BB_LOWER
Date,,,,,,,,,
2006-09-13,1318.07,1312.74,1319.92,1311.12,0.39%,0.003861,1322.519210,1292.719667,1262.920124
2006-09-14,1316.28,1318.00,1318.00,1313.25,-0.14%,-0.001358,1324.434096,1294.015333,1263.596570
2006-09-15,1319.66,1316.28,1324.65,1316.28,0.26%,0.002568,1326.678258,1295.328333,1263.978408
2006-09-18,1321.18,1319.85,1324.87,1318.16,0.12%,0.001152,1328.843769,1296.722333,1264.600898
2006-09-19,1317.64,1321.17,1322.04,1312.17,-0.27%,-0.002679,1330.110596,1298.118000,1266.125404
2006-09-20,1325.18,1318.28,1328.53,1318.28,0.57%,0.005722,1331.742076,1299.908000,1268.073924
2006-09-21,1318.03,1324.89,1328.19,1315.45,-0.54%,-0.005395,1331.429581,1301.644000,1271.858419
2006-09-22,1314.78,1318.03,1318.03,1310.94,-0.25%,-0.002466,1330.999875,1303.076333,1275.152791
2006-09-25,1326.37,1314.78,1329.35,1311.58,0.88%,0.008815,1330.678494,1305.064000,1279.449506


In [123]:
# Creating Signal.
# Set the Signal column
bb_signals_df["Signal"] = 0.0

# Create a value to hold the initial trade signal
trade_signal = 0

# Update the DataFrame Signal column 1 (entry) or -1 (exit) for a long position trading algorithm
# where 1 is when the Close price is less than the BB_LOWER window
# where -1 is when the Close price is greater the the BB_UPPER window
# Incorporate a conditional in the if-statement, to evaluate the value of the trade_signal so the algorithm 
# plots only 1 entry and exit point per cycle.
for index, row in bb_signals_df.iterrows():
    if (row["close"] < row["BB_LOWER"]) and (trade_signal < 1):
        bb_signals_df.loc[index, "Signal"] = 1.0
        trade_signal += 1
        
    if (row["close"] > row["BB_UPPER"]) and (trade_signal > 0):
        bb_signals_df.loc[index, "Signal"] = -1.0
        trade_signal = 0


# Review the DataFrame
bb_signals_df

,close,open,high,low,change %,actual_returns,BB_UPPER,BB_MIDDLE,BB_LOWER,Signal
Date,,,,,,,,,,
2006-07-03,1280.19,1270.06,1280.38,1270.06,0.79%,0.007865,NaN,NaN,NaN,0.0
2006-07-05,1270.91,1280.05,1280.05,1265.91,-0.72%,-0.007249,NaN,NaN,NaN,0.0
2006-07-06,1274.08,1270.58,1278.32,1270.58,0.25%,0.002494,NaN,NaN,NaN,0.0
2006-07-07,1265.48,1274.08,1275.38,1263.13,-0.67%,-0.006750,NaN,NaN,NaN,0.0
2006-07-10,1267.34,1265.46,1274.06,1264.46,0.15%,0.001470,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...
2022-05-05,4152.38,4255.50,4256.39,4105.51,-3.44%,-0.034368,4704.964276,4400.417000,4095.869724,0.0
2022-05-06,4123.34,4128.17,4157.69,4067.91,-0.70%,-0.006994,4704.422307,4387.189667,4069.957027,0.0
2022-05-09,3991.24,4081.27,4081.27,3975.48,-3.20%,-0.032037,4711.594624,4368.796333,4025.998043,0.0


In [124]:
# Visualize entry position relative to close price
entry = bb_signals_df[bb_signals_df["Signal"] == 1.0]["close"].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = bb_signals_df[bb_signals_df["Signal"] == -1.0]["close"].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = bb_signals_df[["close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_upper = bb_signals_df[["BB_UPPER"]].hvplot(
    line_color='purple',
    ylabel='Price in $',
    width=1000,
    height=400
)


bb_middle = bb_signals_df[["BB_MIDDLE"]].hvplot(
    line_color='orange',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_lower = bb_signals_df[["BB_LOWER"]].hvplot(
    line_color='blue',
    ylabel='Price in $',
    width=1000,
    height=400
)


# Overlay plots
bbands_plot = security_close * bb_upper * bb_middle * bb_lower * entry * exit
bbands_plot

:Overlay
   .Curve.I          :Curve   [Date]   (close)
   .Curve.II         :Curve   [Date]   (BB_UPPER)
   .Curve.III        :Curve   [Date]   (BB_MIDDLE)
   .Curve.IV         :Curve   [Date]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [Date]   (close)
   .Scatter.Close.II :Scatter   [Date]   (close)

In [125]:
# Predicting the algorithm returns against actual returns

predictions_df = pd.DataFrame(index=bb_signals_df.index)
predictions_df["predicted_signal"] = bb_signals_df["Signal"]
predictions_df["actual_returns"] = bb_signals_df["actual_returns"]
# Calculate the  strategy returns
predictions_df["strategy_returns"] = predictions_df["actual_returns"] * predictions_df["predicted_signal"].cumsum()
# Calculate the cumulative returns
predictions_df["cumulative_Returns"] = (
    1 + predictions_df["strategy_returns"]
).cumprod() -1

predictions_df.head()

,predicted_signal,actual_returns,strategy_returns,cumulative_Returns
Date,,,,
2006-07-03,0.0,0.007865,0.0,0.0
2006-07-05,0.0,-0.007249,-0.0,0.0
2006-07-06,0.0,0.002494,0.0,0.0
2006-07-07,0.0,-0.006750,-0.0,0.0
2006-07-10,0.0,0.001470,0.0,0.0


In [126]:
portfolio_evaluation_df = lib.performance_metrics(predictions_df, "Strategy_Bollinger_L_Term")

portfolio_evaluation_df.head()

,Strategy_Bollinger_L_Term
Annualized Return,0.015818
Cumulative Returns,-0.009339
Annual Volatility,2.137405
Sharpe Ratio,0.087436
Sortino Ratio,0.120971


In [127]:
(1 + predictions_df[["actual_returns", "strategy_returns"]]).cumprod().hvplot()

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [128]:
# writing into file for comparison with different models.

portfolio_evaluation_df.to_csv("../data/Strategy_Bollinger_L_Term_Metrics.csv")

predictions_df.to_csv("../data/Strategy_Bollinger_L_Term_Returns.csv")